In [ ]:
!pip install -q pandas==2.2.2
!pip install -q -U google-generativeai
!pip install -q -U matplotlib
!pip install -q -U seaborn
!pip install -q -U tabulate
!pip install -q -U langchain
!pip install -q -U langchain-community
!pip install -q -U langchain-experimental
!pip install -q -U "langchain-core>=1.0.0"
!pip install -q -U langchain-google-genai
!pip install -q -U tenacity
!pip install -q -U packaging

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from google.colab import userdata
import google.generativeai as genai
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_google_genai import ChatGoogleGenerativeAI

sns.set_theme(style="whitegrid")

In [ ]:
try:

    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("API Key successfully loaded.")

except Exception as e:

    print("Error loading API Key. Please make sure 'GOOGLE_API_KEY' is set in Colab Secrets.")

In [ ]:
from google.colab import files

print("Please upload your CSV file:")
uploaded = files.upload()
filename = next(iter(uploaded))

try:

    df = pd.read_csv(filename)
    print(f"\nSuccessfully loaded '{filename}'")
    print("Data Preview:")
    display(df.head())

except Exception as e:

    print(f"Error loading CSV: {e}")

In [ ]:
for m in genai.list_models():
  if "generateContent" in m.supported_generation_methods:
    print(m.name)

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-flash-lite-latest",
    temperature=0,
    max_output_tokens=2048
)

agent = create_pandas_dataframe_agent(
    llm,
    df,
    verbose=False,
    allow_dangerous_code=True
)

print("Agent initialized successfully! You can now ask questions.")

In [ ]:
def ask_agent(query):

    print(f"Processing Query: '{query}'...\n")

    try:
        response = agent.run(query)
        print("\nAnswer:")
        print(response)

        if plt.get_fignums():
            plt.show()
        else:
            pass

    except Exception as e:
        print(f"An error occurred: {e}")

query = "Show me a distribution of the numerical columns"
ask_agent(query)

In [ ]:
while True:

    user_input = input("\nAsk a question (or type 'exit' to quit): ")

    if user_input.lower() in ['exit', 'quit']:
        break

    ask_agent(user_input)